In [1]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_size, n_layers, hidden_size, output_size):
        super(PINN, self).__init__()
        self.input_size = input_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.layers = nn.ModuleList([nn.Linear(self.input_size, self.hidden_size)])
        self.layers.extend([nn.Linear(self.hidden_size, self.hidden_size) for _ in range(self.n_layers - 2)])
        self.layers.append(nn.Linear(self.hidden_size, self.output_size))
        self.activation = nn.Tanh()
        self.init_weights()
    
    def init_weights(self):
        for layer in self.layers:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)
        return x

In [2]:
freq_vectors_shg1 = np.load("Data/shg_freq_domain_ds.npy")
freq_vectors_shg2 = freq_vectors_shg1  # these are equivalent here
freq_vectors_sfg = np.load("Data/sfg_freq_domain_ds.npy")

sfg_original_freq = np.load("Data/sfg_original_freq_vector.npy")
sfg_original_time = np.load("Data/sfg_original_time_vector.npy")
sfg_original_time_ds = sfg_original_time[1] - sfg_original_time[0]

In [6]:
freq_vectors_sfg * 1e-12

array([-3.60477332, -3.59182647, -3.57887962, -3.56593277, -3.55298591,
       -3.54003906, -3.52709221, -3.51414536, -3.50119851, -3.48825166,
       -3.47530481, -3.46235795, -3.4494111 , -3.43646425, -3.4235174 ,
       -3.41057055, -3.3976237 , -3.38467685, -3.37173   , -3.35878314,
       -3.34583629, -3.33288944, -3.31994259, -3.30699574, -3.29404889,
       -3.28110204, -3.26815518, -3.25520833, -3.24226148, -3.22931463,
       -3.21636778, -3.20342093, -3.19047408, -3.17752723, -3.16458037,
       -3.15163352, -3.13868667, -3.12573982, -3.11279297, -3.09984612,
       -3.08689927, -3.07395241, -3.06100556, -3.04805871, -3.03511186,
       -3.02216501, -3.00921816, -2.99627131, -2.98332446, -2.9703776 ,
       -2.95743075, -2.9444839 , -2.93153705, -2.9185902 , -2.90564335,
       -2.8926965 , -2.87974964, -2.86680279, -2.85385594, -2.84090909,
       -2.82796224, -2.81501539, -2.80206854, -2.78912169, -2.77617483,
       -2.76322798, -2.75028113, -2.73733428, -2.72438743, -2.71

In [3]:
from Analysis.util import change_domains, re_im_combined
import pickle
import os
import h5py
from types import SimpleNamespace
from Utilz.loads import get_datasets

In [4]:
data_directory = "/mnt/oneterra/SFG_reIm_h5/"

In [5]:
args = SimpleNamespace(data_dir=data_directory, train_load_mode=0, val_load_mode=0, test_load_mode=1, crystal_length=100)
train_data, val_data, test_data = get_datasets(args)

In [6]:
train_data_0  = train_data[0]
# to give a fake batch size
train_data_x_0 = train_data_0[0].unsqueeze(0)
train_data_y_0 = train_data_0[1].unsqueeze(0)

In [15]:
train_data_x_0.shape

torch.Size([1, 10, 8264])

In [7]:
one_sample = train_data_x_0[0, 0]

In [13]:
shg1, shg2, sfg = re_im_combined(one_sample)
sfg = sfg.cpu().numpy() 
shg1 = shg1.cpu().numpy()
shg2 = shg2.cpu().numpy()

In [14]:
_, resampled_sfg = change_domains(domain=freq_vectors_sfg, field=sfg, new_domain=sfg_original_freq, domain_type="non")
_, resampled_shg1 = change_domains(domain=freq_vectors_shg1, field=shg1, new_domain=sfg_original_freq, domain_type="non")
_, resampled_shg2 = change_domains(domain=freq_vectors_shg2, field=shg2, new_domain=sfg_original_freq, domain_type="non")

field type not supported
field type not supported
field type not supported


In [15]:
resampled_sfg.shape, resampled_shg1.shape, resampled_shg2.shape

((32768,), (32768,), (32768,))